In [3]:
import fitz
from PIL import Image
import os

In [17]:
def pdf_to_images(pdf_path,output_folder):
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    images = fitz.open(pdf_path)
    image_paths = []

    for i,image in enumerate(images):
        page = images.load_page(i)
        pix = page.get_pixmap()
        image_path = os.path.join(output_folder,f"page{i+1}.jpg")
        pix.save(image_path)
        image_paths.append(image_path)

    images.close()

    return image_paths

In [18]:
pdf_path = "books\Things mother used to make.pdf"
output_folder = 'images'
image_paths = pdf_to_images(pdf_path,output_folder)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API')

In [5]:
from groq import Groq
import base64

In [6]:
client = Groq()
MODEL = 'meta-llama/llama-4-scout-17b-16e-instruct'

In [8]:
from PIL import ImageFile


image_path = "images/page23.jpg"
with open(image_path,'rb') as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')
image_data

'...'

In [9]:
system_prompt = """
please analyze the content of this image and extract any related recipe information
"""

In [10]:
response = client.chat.completions.create(
        model = MODEL,
        messages = [
            {'role':'system','content':system_prompt},
            {'role':'user','content':[
                {'type':'text',
                'text':'This is the image from the recipe page.'},
                {'type':'image_url',
                'image_url':{'url':f"data:image/jpeg;base64,{image_data}",
                'detail':'low'}}
            ]}
        ]
    )

In [11]:
from IPython.display import display,Markdown

In [7]:
def get_response():
    ans = response.choices[0].message.content
    return display(Markdown(ans))


In [ ]:
get_response()

In [13]:
system_prompt2 = """
please analyze the content of this image and extract any related recipe information into structure components.
specifically extract the recipe title, list of ingredients, step by step instructions, cuisine type, dish type, any relevant tags or metadata.  
the output must be formatted in a way suited for embedding in a Retrieval augmented generation system and if you not find any recipe information just print `no recipe found` nothing more than that
"""

In [14]:
response = client.chat.completions.create(
        model = MODEL,
        messages = [
            {'role':'system','content':system_prompt2},
            {'role':'user','content':[
                {'type':'text',
                'text':'This is the image from the recipe page.'},
                {'type':'image_url',
                'image_url':{'url':f"data:image/jpeg;base64,{image_data}",
                'detail':'low'}}
            ]}
        ],
        temperature = 0,
    )

In [15]:
get_response()

Here are the extracted recipe components:

**Recipe 1: Bannocks**

* **Recipe Title:** Bannocks
* **Cuisine Type:** Traditional/Scottish
* **Dish Type:** Bread
* **Tags:** Scottish, Bread, Traditional
* **Ingredients:**
	+ 1 Cupful of Thick Sour Milk
	+ 2 Cupfuls of Flour
	+ 1/2 Cupful of Sugar
	+ 1 Teaspoonful of Soda
	+ 1 Egg
	+ A pinch of Salt
* **Instructions:**
	1. Make the mixture stiff enough to drop from a spoon.
	2. Drop mixture, size of a walnut, into boiling fat.
	3. Serve warm, with maple syrup.

**Recipe 2: Boston Brown Bread**

* **Recipe Title:** Boston Brown Bread
* **Cuisine Type:** American
* **Dish Type:** Bread
* **Tags:** American, Bread, Traditional
* **Ingredients:**
	+ 1 Cupful of Rye Meal
	+ 1 Cupful of Sour Milk
	+ 1 Cupful of Graham Meal
	+ 1 Cupful of Molasses
	+ 1/2 Cupful of Flour
	+ 1/2 Teaspoonful of Salt
	+ 1 Cupful of Indian Meal
	+ 1 Cupful of Sweet Milk
	+ 1 Heaping Teaspoonful of Soda
* **Instructions:**
	1. Stir the meals and salt together.
	2. Beat the soda into the molasses until it foams; add sour milk, mix all together and pour into a tin pail which has been well greased, if you have no brown-bread steamer.

In [19]:
extracted_recipes =[]

for image_path in image_paths:
    with open(image_path,'rb') as image_file:
        image_data = base64.b64encode(image_file.read()).decode('utf-8')

    response = client.chat.completions.create(
        model = MODEL,
        messages = [
            {'role':'system','content':system_prompt2},
            {'role':'user','content':[
                {'type':'text',
                'text':'This is the image from the recipe page.'},
                {'type':'image_url',
                'image_url':{'url':f"data:image/jpeg;base64,{image_data}",
                'detail':'low'}}
            ]}
        ],
        temperature=0,
    )

    ans = response.choices[0].message.content
    extracted_recipes.append({'image_path':image_path,'recipe_info':ans})
    print(f"Extracted information for {image_path}:\n{ans}\n")


Extracted information for images\page1.jpg:
No recipe found

Extracted information for images\page2.jpg:
no recipe found

Extracted information for images\page3.jpg:
no recipe found

Extracted information for images\page4.jpg:
No recipe found

Extracted information for images\page5.jpg:
No recipe found

Extracted information for images\page6.jpg:
No recipe found

Extracted information for images\page7.jpg:
No recipe found

Extracted information for images\page8.jpg:
No recipe found

Extracted information for images\page9.jpg:
No recipe found

Extracted information for images\page10.jpg:
no recipe found

Extracted information for images\page11.jpg:
No recipe found

Extracted information for images\page12.jpg:
no recipe found

Extracted information for images\page13.jpg:
No recipe found

Extracted information for images\page14.jpg:
No recipe found

Extracted information for images\page15.jpg:
No recipe found

Extracted information for images\page16.jpg:
No recipe found

Extracted informa

In [20]:
extracted_recipes

[{'image_path': 'images\\page1.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page2.jpg', 'recipe_info': 'no recipe found'},
 {'image_path': 'images\\page3.jpg', 'recipe_info': 'no recipe found'},
 {'image_path': 'images\\page4.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page5.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page6.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page7.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page8.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page9.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page10.jpg', 'recipe_info': 'no recipe found'},
 {'image_path': 'images\\page11.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page12.jpg', 'recipe_info': 'no recipe found'},
 {'image_path': 'images\\page13.jpg', 'recipe_info': 'No recipe found'},
 {'image_path': 'images\\page14.jpg', 'recipe_info': 'No rec

In [9]:
import json

In [21]:
filtered_recipes = []
for recipe in extracted_recipes:
    if any(keyword in recipe['recipe_info'].lower() for keyword in ['ingredients','instructions','recipe title']):
        filtered_recipes.append(recipe)
    else:
        print(f"skipping recipe: {recipe['image_path']}")

skipping recipe: images\page1.jpg
skipping recipe: images\page2.jpg
skipping recipe: images\page3.jpg
skipping recipe: images\page4.jpg
skipping recipe: images\page5.jpg
skipping recipe: images\page6.jpg
skipping recipe: images\page7.jpg
skipping recipe: images\page8.jpg
skipping recipe: images\page9.jpg
skipping recipe: images\page10.jpg
skipping recipe: images\page11.jpg
skipping recipe: images\page12.jpg
skipping recipe: images\page13.jpg
skipping recipe: images\page14.jpg
skipping recipe: images\page15.jpg
skipping recipe: images\page16.jpg
skipping recipe: images\page17.jpg
skipping recipe: images\page18.jpg
skipping recipe: images\page19.jpg
skipping recipe: images\page20.jpg
skipping recipe: images\page21.jpg
skipping recipe: images\page22.jpg
skipping recipe: images\page106.jpg
skipping recipe: images\page107.jpg
skipping recipe: images\page108.jpg
skipping recipe: images\page109.jpg
skipping recipe: images\page110.jpg
skipping recipe: images\page111.jpg
skipping recipe: images

In [25]:
output_file = 'recipe_info.json'

with open(output_file,'w') as json_file:
    json.dump(filtered_recipes,json_file)

In [10]:
import numpy as np

In [11]:
with open('recipe_info.json','r') as json_file:
    filtered_recipes = json.load(json_file)

In [12]:
os.environ['NVIDIA_API_KEY']=os.getenv('NVIDIA_TOKEN')

In [13]:
recipe_texts = [recipe['recipe_info'] for recipe in filtered_recipes]

In [14]:
recipe_texts

['Here are the extracted recipe components:\n\n**Recipe 1: Bannocks**\n\n* **Recipe Title:** Bannocks\n* **Cuisine Type:** Traditional/Scottish\n* **Dish Type:** Bread\n* **Tags:** Scottish, Bread, Traditional\n* **Ingredients:**\n\t+ 1 Cupful of Thick Sour Milk\n\t+ 2 Cupfuls of Flour\n\t+ 1/2 Cupful of Sugar\n\t+ 1 Teaspoonful of Soda\n\t+ 1 Egg\n\t+ A pinch of Salt\n* **Instructions:**\n\t1. Make the mixture stiff enough to drop from a spoon.\n\t2. Drop mixture, size of a walnut, into boiling fat.\n\t3. Serve warm, with maple syrup.\n\n**Recipe 2: Boston Brown Bread**\n\n* **Recipe Title:** Boston Brown Bread\n* **Cuisine Type:** American\n* **Dish Type:** Bread\n* **Tags:** American, Bread, Traditional\n* **Ingredients:**\n\t+ 1 Cupful of Rye Meal\n\t+ 1 Cupful of Sour Milk\n\t+ 1 Cupful of Graham Meal\n\t+ 1 Cupful of Molasses\n\t+ 1/2 Cupful of Flour\n\t+ 1/2 Teaspoonful of Salt\n\t+ 1 Cupful of Indian Meal\n\t+ 1 Cupful of Sweet Milk\n\t+ 1 Heaping Teaspoonful of Soda\n* **Instr

In [15]:
len(recipe_texts)

83

In [16]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.docstore.document import Document

In [17]:
embedding = NVIDIAEmbeddings(model = "nvidia/nv-embed-v1")

In [18]:
docs = [Document(page_content=text) for text in recipe_texts]

In [19]:
docs

[Document(metadata={}, page_content='Here are the extracted recipe components:\n\n**Recipe 1: Bannocks**\n\n* **Recipe Title:** Bannocks\n* **Cuisine Type:** Traditional/Scottish\n* **Dish Type:** Bread\n* **Tags:** Scottish, Bread, Traditional\n* **Ingredients:**\n\t+ 1 Cupful of Thick Sour Milk\n\t+ 2 Cupfuls of Flour\n\t+ 1/2 Cupful of Sugar\n\t+ 1 Teaspoonful of Soda\n\t+ 1 Egg\n\t+ A pinch of Salt\n* **Instructions:**\n\t1. Make the mixture stiff enough to drop from a spoon.\n\t2. Drop mixture, size of a walnut, into boiling fat.\n\t3. Serve warm, with maple syrup.\n\n**Recipe 2: Boston Brown Bread**\n\n* **Recipe Title:** Boston Brown Bread\n* **Cuisine Type:** American\n* **Dish Type:** Bread\n* **Tags:** American, Bread, Traditional\n* **Ingredients:**\n\t+ 1 Cupful of Rye Meal\n\t+ 1 Cupful of Sour Milk\n\t+ 1 Cupful of Graham Meal\n\t+ 1 Cupful of Molasses\n\t+ 1/2 Cupful of Flour\n\t+ 1/2 Teaspoonful of Salt\n\t+ 1 Cupful of Indian Meal\n\t+ 1 Cupful of Sweet Milk\n\t+ 1 Hea

In [20]:
embedding_response = embedding.embed_documents(recipe_texts)

In [21]:
embedding_response

[[-0.015960693359375,
  -0.00650787353515625,
  0.030487060546875,
  -0.00955963134765625,
  0.0234375,
  0.016815185546875,
  0.005504608154296875,
  0.007282257080078125,
  -0.0150146484375,
  0.00021827220916748047,
  0.010955810546875,
  -0.007289886474609375,
  -0.0103912353515625,
  -0.00931549072265625,
  -0.0154266357421875,
  0.0029449462890625,
  0.003391265869140625,
  -0.0103607177734375,
  -0.0016927719116210938,
  0.00687408447265625,
  0.0046539306640625,
  -0.01132965087890625,
  -0.00284576416015625,
  0.0057373046875,
  -0.0033721923828125,
  0.004638671875,
  0.004497528076171875,
  0.0006275177001953125,
  0.001880645751953125,
  0.0103912353515625,
  0.0066070556640625,
  0.01200103759765625,
  0.0097808837890625,
  0.00875091552734375,
  0.0089111328125,
  0.0027561187744140625,
  -0.0038909912109375,
  0.01812744140625,
  0.00954437255859375,
  0.0309600830078125,
  0.0094146728515625,
  -0.004608154296875,
  0.01380157470703125,
  -0.0030727386474609375,
  -0.01

In [22]:
embedding_matrix = np.array(embedding_response)
embedding_matrix

array([[-0.01596069, -0.00650787,  0.03048706, ..., -0.00147533,
         0.00174236, -0.00597   ],
       [-0.0057869 , -0.02183533,  0.02319336, ..., -0.01552582,
        -0.02418518, -0.02870178],
       [-0.01792908,  0.00138474,  0.03039551, ..., -0.00333595,
        -0.00554657, -0.02456665],
       ...,
       [-0.01675415, -0.0166626 ,  0.01329803, ..., -0.00049877,
        -0.0107193 , -0.00920105],
       [ 0.00019288, -0.02905273,  0.02859497, ..., -0.00017238,
         0.00255775, -0.00635147],
       [ 0.00505066, -0.01359558,  0.02947998, ...,  0.0198822 ,
        -0.0021534 , -0.00621414]])

In [23]:
len(embedding_matrix),len(embedding_response[0])   

(83, 4096)

In [24]:
embedding_matrix.shape

(83, 4096)

In [25]:
import faiss

In [26]:
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

In [27]:
faiss.write_index(index,'fitered_recipe_index.index')

In [28]:
metadata = [{'recipe_info':recipe['recipe_info'],
             'image_path':recipe['image_path']} for recipe in filtered_recipes]
metadata

[{'recipe_info': 'Here are the extracted recipe components:\n\n**Recipe 1: Bannocks**\n\n* **Recipe Title:** Bannocks\n* **Cuisine Type:** Traditional/Scottish\n* **Dish Type:** Bread\n* **Tags:** Scottish, Bread, Traditional\n* **Ingredients:**\n\t+ 1 Cupful of Thick Sour Milk\n\t+ 2 Cupfuls of Flour\n\t+ 1/2 Cupful of Sugar\n\t+ 1 Teaspoonful of Soda\n\t+ 1 Egg\n\t+ A pinch of Salt\n* **Instructions:**\n\t1. Make the mixture stiff enough to drop from a spoon.\n\t2. Drop mixture, size of a walnut, into boiling fat.\n\t3. Serve warm, with maple syrup.\n\n**Recipe 2: Boston Brown Bread**\n\n* **Recipe Title:** Boston Brown Bread\n* **Cuisine Type:** American\n* **Dish Type:** Bread\n* **Tags:** American, Bread, Traditional\n* **Ingredients:**\n\t+ 1 Cupful of Rye Meal\n\t+ 1 Cupful of Sour Milk\n\t+ 1 Cupful of Graham Meal\n\t+ 1 Cupful of Molasses\n\t+ 1/2 Cupful of Flour\n\t+ 1/2 Teaspoonful of Salt\n\t+ 1 Cupful of Indian Meal\n\t+ 1 Cupful of Sweet Milk\n\t+ 1 Heaping Teaspoonful of

In [63]:
with open('recipe_metadata.json','w') as json_file:
    json.dump(metadata,json_file,indent=4)

In [70]:
k=5
query = 'how to make a bread'
query_embedding = embedding.embed_query(query)
print(f"query embedding is {query_embedding}\n")
query_vector = np.array(query_embedding).reshape(1,-1)
print(f"query vector is {query_vector}\n")

distances,indices = index.search(query_vector,min(k,len(metadata)))
print(f"the distances are {distances}\n")
print(f"the indices are {indices}\n")

stored_indecis = indices[0].tolist()
stored_distances = distances[0].tolist()

print('The metadata content is')
for i,dist in zip(stored_indecis,stored_distances):
    if 0<=i<len(metadata):
        print(f"distance: {dist}, metadata: {metadata[i]['recipe_info']}")

query embedding is [-0.01438140869140625, 0.000980377197265625, 0.01080322265625, 0.0035572052001953125, -0.0357666015625, 0.0159759521484375, -0.0041351318359375, 0.0012006759643554688, -0.005573272705078125, 0.022216796875, -0.0071563720703125, 0.00559234619140625, -0.0101470947265625, -0.0059661865234375, -0.0092620849609375, -0.005718231201171875, -0.006687164306640625, -0.005672454833984375, 0.00666046142578125, 0.01042938232421875, -0.00044035911560058594, 0.0018329620361328125, 0.004688262939453125, 0.01611328125, -0.0007214546203613281, -0.031402587890625, -0.0020599365234375, 0.0323486328125, 0.037078857421875, 0.00789642333984375, -0.0294342041015625, 0.00286102294921875, -0.0058746337890625, -0.00727081298828125, 0.0072174072265625, 0.0183258056640625, -0.002490997314453125, 0.0281219482421875, -0.02423095703125, 0.0335693359375, 0.0140380859375, 0.005023956298828125, -0.00396728515625, 0.00847625732421875, -0.01094818115234375, 0.0014581680297851562, -0.00725555419921875, -

In [ ]:
results=[]
for i,dist in zip(stored_indecis,stored_distances):
    if 0<=i<len(metadata):
        results.append((metadata[i]['recipe_info'],dist))


In [84]:
display(Markdown(results[4][0]))

Here are the extracted recipe components:

**Recipes:**

### Brown Bread (Baked)

* **Recipe Title:** Brown Bread (Baked)
* **Cuisine Type:** Traditional/ Classic
* **Dish Type:** Bread
* **Tags:** baked, bread, traditional

* **Ingredients:**
  + 1 Cupful of Indian Meal
  + 1 Cupful of Molasses (scant)
  + 1 Cupful of Rye Meal
  + 1/2 Cupful of Flour
  + 1 Teaspoonful of Soda
  + 1 Cupful of Milk or Water

* **Instructions:**
  1. Put the meals and flour together. 
  2. Stir soda into molasses until it foams.
  3. Add salt and milk or water. 
  4. Mix all together. 
  5. Bake in a tin pan with cover on for two and a half hours.

### Coffee Cakes

* **Recipe Title:** Coffee Cakes
* **Cuisine Type:** Traditional/ Classic
* **Dish Type:** Cakes
* **Tags:** cakes, traditional

* **Ingredients:**
  (No ingredients listed)

* **Instructions:**
  1. When your dough for yeast bread is risen light and fluffy, cut off small pieces and roll as big as your finger, four inches long. 
  2. Fold and twist to two inches long and fry in deep fat. 
  3. Serve hot with coffee.

### Corn Meal Gems

* **Recipe Title:** Corn Meal Gems
* **Cuisine Type:** Traditional/ Classic
* **Dish Type:** Gems/ Muffins
* **Tags:** gems, muffins, traditional

* **Ingredients:**
  + 2 Cupfuls of Flour
  + 1 Teaspoonful of Baking Soda
  + 1 Cupful of Corn Meal (bolted is best)
  + 1 Egg
  + 2 Cupfuls of Milk
  + 2 1/2 Cupful of Sugar
  + 2 Teaspoonfuls of Cream of Tartar
  + 1/2 Teaspoonful of Salt

* **Instructions:**
  1. Stir the flour and meal together, adding cream of tartar, soda, salt and sugar. 
  2. Beat the egg, add the (rest of instruction cut off)

In [29]:
def query_embeddings(query,index,metadata,k=5):
    query_embedding = embedding.embed_query(query)
    #print(f"query embedding is {query_embedding}\n")
    query_vector = np.array(query_embedding).reshape(1,-1)
    #print(f"query vector is {query_vector}\n")

    distances,indices = index.search(query_vector,min(k,len(metadata)))
    #print(f"the distances are {distances}\n")
    #print(f"the indices are {indices}\n")

    stored_indecis = indices[0].tolist()
    stored_distances = distances[0].tolist()

    # print('The metadata content is')
    # for i,dist in zip(stored_indecis,stored_distances):
    #     if 0<=i<len(metadata):
    #         print(f"distance: {dist}, metadata: {metadata[i]['recipe_info']}")

    results =[(metadata[i]['recipe_info'],dist) for i,dist in zip(
                stored_indecis,stored_distances) if 0<=i<len(metadata)]

    return results

In [67]:
query = 'chocolate cake'
result = query_embeddings(query,index,metadata)
print(f'The result are {result}')

The result are [('Here are the extracted recipe components:\n\n**Recipes Found:**\n\n### Bread Pudding\n\n* **Recipe Title:** Bread Pudding\n* **Cuisine Type:** Traditional/ Classic\n* **Dish Type:** Dessert\n* **Tags:** Pudding, Bread, Dessert\n* **Ingredients:**\n\t+ 1 Pint of Stale Bread\n\t+ 1 Quart of Milk\n\t+ 1 Cupful of Sugar\n\t+ 1 Egg\n\t+ 1/2 Cupful of Raisins\n\t+ 1 Teaspoonful of Cinnamon\n\t+ 1/2 Teaspoonful of Salt\n* **Instructions:**\n\t1. Pour hot water over the stale bread and let soak until soft.\n\t2. Then add other ingredients and bake for three hours in a moderate oven.\n\t3. If eaten cold, serve with hot sauce. If eaten hot, serve with cold sauce.\n\n### Steamed Chocolate Pudding\n\n* **Recipe Title:** Steamed Chocolate Pudding\n* **Cuisine Type:** Traditional/ Classic\n* **Dish Type:** Dessert\n* **Tags:** Pudding, Chocolate, Dessert\n* **Ingredients:**\n\t+ Butter size of a Walnut\n\t+ 1/2 Cupful of Sugar\n\t+ 1/2 Cupful of Milk\n\t+ 1 Cupful of Flour\n\t+ 1 T

In [31]:
def combined_retrived_content(results):
    combined_content = "\n\n".join([result[0] for result in results])
    return combined_content

In [68]:
combined_content = combined_retrived_content(result)
print(f"The combined content is {combined_content}")

The combined content is Here are the extracted recipe components:

**Recipes Found:**

### Bread Pudding

* **Recipe Title:** Bread Pudding
* **Cuisine Type:** Traditional/ Classic
* **Dish Type:** Dessert
* **Tags:** Pudding, Bread, Dessert
* **Ingredients:**
	+ 1 Pint of Stale Bread
	+ 1 Quart of Milk
	+ 1 Cupful of Sugar
	+ 1 Egg
	+ 1/2 Cupful of Raisins
	+ 1 Teaspoonful of Cinnamon
	+ 1/2 Teaspoonful of Salt
* **Instructions:**
	1. Pour hot water over the stale bread and let soak until soft.
	2. Then add other ingredients and bake for three hours in a moderate oven.
	3. If eaten cold, serve with hot sauce. If eaten hot, serve with cold sauce.

### Steamed Chocolate Pudding

* **Recipe Title:** Steamed Chocolate Pudding
* **Cuisine Type:** Traditional/ Classic
* **Dish Type:** Dessert
* **Tags:** Pudding, Chocolate, Dessert
* **Ingredients:**
	+ Butter size of a Walnut
	+ 1/2 Cupful of Sugar
	+ 1/2 Cupful of Milk
	+ 1 Cupful of Flour
	+ 1 Teaspoonful of Baking- powder
	+ 1 Square of

In [33]:
from langchain_nvidia import ChatNVIDIA

In [77]:
model = ChatNVIDIA(model = "meta/llama-3.1-405b-instruct",temperature = 0)

In [78]:
from langchain_core.messages import SystemMessage,HumanMessage

In [79]:
system_prompt3 = f'''
you are highly experienced and expert chef specialised in providing cooking advice.
your main task is to provide information precise and accurate on the combined content that may or may not include multiple recipes
you answer directly to the query using only information from the provided {combined_content}.
if you don't know the answer, just say that you don't know.
your goal is to help the user and answer the {query}
'''

In [94]:
strr=''
for chunk in model.stream([{"role":"system","content":system_prompt3},
                            {'role':'user','content':'how to make a cake'},
                            {'role':'assistant','content':combined_content}]): 
  print(chunk.content, end="")
  strr+=chunk.content

I can provide you with a recipe for a cake. Here are a few options:

**Recipe 1: One, Two, Three, Four Cake**

* **Ingredients:**
	+ 1 Cupful of Butter
	+ 2 Cupfuls of Sugar
	+ 3 Cupfuls of Flour
	+ 4 Eggs
	+ 2 1/2 Cupful of Milk
	+ 2 Teaspoons of Cream of Tartar
	+ 1 Teaspoon of Soda
* **Instructions:**
	1. Cream the butter and sugar together and add the well-beaten eggs; beat all and add milk; beat again.
	2. Sift the cream of tartar and the soda into the flour; stir all together.
	3. Bake in a slow oven. This will make two loaves.

**Recipe 2: Ribbon Cake**

* **Ingredients:**
	+ 3 Eggs
	+ 2 Cupfuls of Sugar
	+ 2 1/2 Cupful of Butter
	+ 1 Cupful of Milk
	+ 3 Cupfuls of Flour
	+ 1 Teaspoon of Cream of Tartar
	+ 1 Tablespoon of Molasses
	+ A little Salt and flavor, Lemon or Almond
	+ 1 Large Cupful of Raisins
	+ 1/4 Pound of Citron
	+ 1 Teaspoon of Cinnamon and Cloves
	+ A little Nutmeg
	+ 1/2 Teaspoon of Soda
* **Instructions:**
	1. Cream the butter and sugar together, and add the we

In [81]:
display(Markdown(strr))

I can provide you with a recipe for a cake. Here are a few options:

**Recipe 1: One, Two, Three, Four Cake**

* **Ingredients:**
	+ 1 Cupful of Butter
	+ 2 Cupfuls of Sugar
	+ 3 Cupfuls of Flour
	+ 4 Eggs
	+ 2 1/2 Cupful of Milk
	+ 2 Teaspoons of Cream of Tartar
	+ 1 Teaspoon of Soda
* **Instructions:**
	1. Cream the butter and sugar together and add the well-beaten eggs; beat all and add milk; beat again.
	2. Sift the cream of tartar and the soda into the flour; stir all together.
	3. Bake in a slow oven. This will make two loaves.

**Recipe 2: Ribbon Cake**

* **Ingredients:**
	+ 3 Eggs
	+ 2 Cupfuls of Sugar
	+ 2 1/2 Cupful of Butter
	+ 1 Cupful of Milk
	+ 3 Cupfuls of Flour
	+ 1 Teaspoon of Cream of Tartar
	+ 1 Tablespoon of Molasses
	+ A little Salt and flavor, Lemon or Almond
	+ 1 Large Cupful of Raisins
	+ 1/4 Pound of Citron
	+ 1 Teaspoon of Cinnamon and Cloves
	+ A little Nutmeg
	+ 1/2 Teaspoon of Soda
* **Instructions:**
	1. Cream the butter and sugar together, and add the well-beaten eggs and the milk. Mix the salt, soda and cream of tartar, with the flour. Stir all together.
	2. Put half of this mixture into two oblong pans. To the remainder add one tablespoonful of molasses, one large cupful of raisins, stoned and chopped, a 

**Recipe 3: Roll Jelly Cake**

* **Ingredients:**
	+ 4 Eggs
	+ 1 Cupful of Sugar
	+ 1 Cupful of Flour
	+ 1/2 Teaspoonful of Soda
	+ Pinch of Salt
	+ 1 Teaspoonful of Cream of Tartar
	+ 1 Teaspoonful of Extract of Lemon
* **Instructions:**
	1. Beat together eggs and sugar, add salt and extract.
	2. Stir into the dry flour the soda and cream of tartar. Mix all together.
	3. Bake in a moderate oven, in a large pan, and turn out, when done, on a clean towel, which has been sprinkled with powdered sugar. Spread with jelly and roll while warm.

**Recipe 4: Silver Cake**

* **Ingredients:**
	+ 1 Cupful of Sugar
	+ 1/2 Cupful of Butter
	+ 2 Cupfuls of Flour
	+ Whites of 3 Eggs
	+ 1/2 Teaspoonful of Soda
	+ 1/2 Cupful of Milk
	+ 1 Scant Teaspoonful of Cream of Tartar
	+ Almond Flavoring
* **Instructions:**
	1. Cream together the butter and sugar, add milk and flavoring.
	2. Stir cream of tartar and soda into dry flour. Last of all add whites of eggs, beaten to a 

Please let me know which recipe you would like to use, or if you have any specific preferences (e.g. chocolate cake).

In [47]:
messages=[
    SystemMessage(content = system_prompt3),
    HumanMessage(content = 'how to make a bread')
]

In [ ]:
from IPython.display import Markdown,display

In [49]:
response = model.invoke(messages)
display(Markdown(response.content))

Making bread is a wonderful process. There are several ways to make bread, and I'll provide a general outline of a basic bread recipe. I'll also highlight a few different methods from the recipes I have available.

**Basic Bread Recipe:**

Ingredients:

* 2-3 cupfuls of flour
* 1/2 teaspoonful of salt
* 1/4 teaspoonful of sugar
* 1/2 teaspoonful of active dry yeast (or 1/2 yeast cake)
* 1 cupful of warm water (or milk)

Instructions:

1. Mix the dry ingredients (flour, salt, sugar) together in a bowl.
2. Add the yeast to the warm water (or milk) and let it dissolve.
3. Add the yeast mixture to the dry ingredients and mix until a dough forms.
4. Knead the dough for 10-15 minutes until it becomes smooth and elastic.
5. Place the dough in a greased bowl, cover it, and let it rise in a warm, draft-free place for about an hour, or until it has doubled in size.
6. Preheat your oven to 375°F (190°C).
7. Punch down the dough and shape it into a loaf.
8. Place the loaf in a greased loaf pan and bake for 30-40 minutes, or until the bread is golden brown and sounds hollow when tapped on the bottom.

**Alternative Methods:**

* From the "Nut Bread" recipe, you can add chopped nuts to the dough for added flavor and texture.
* From the "Oatmeal Bread" recipe, you can use rolled oats and boiling water to create a unique bread.
* From the "White Bread" recipe, you can use a simpler method that involves rubbing sugar, salt, and lard into the flour, then adding yeast and water.
* From the "Brown Bread (Baked)" recipe, you can use a mixture of Indian meal, molasses, and rye meal to create a denser, darker bread.

These are just a few examples, and there are many more variations and techniques to explore in the world of bread-making!

In [90]:
def generate_response(query,combined_content,system_prompt):
    strr=''
    for chunk in model.stream([{"role":"system","content":system_prompt},
                                {'role':'user','content':query},
                                {'role':'assistant','content':combined_content}]): 
        #print(chunk.content, end="")
        strr+=chunk.content
    return strr


In [91]:
query = 'how to make cake'
#combined_content = combined_retrived_content(result)
response = generate_response(query,combined_content,system_prompt3)

In [92]:
display(Markdown(response))

I can provide you with a few cake recipes. Here are the ingredients and instructions for a few different types of cakes:

**One, Two, Three, Four Cake**

* **Ingredients:**
	+ 1 Cupful of Butter
	+ 2 Cupfuls of Sugar
	+ 3 Cupfuls of Flour
	+ 4 Eggs
	+ 2 1/2 Cupful of Milk
	+ 2 Teaspoons of Cream of Tartar
	+ 1 Teaspoon of Soda
* **Instructions:**
	1. Cream the butter and sugar together and add the well-beaten eggs; beat all and add milk; beat again.
	2. Sift the cream of tartar and the soda into the flour; stir all together.
	3. Bake in a slow oven. This will make two loaves.

**Ribbon Cake**

* **Ingredients:**
	+ 3 Eggs
	+ 2 Cupfuls of Sugar
	+ 2 1/2 Cupful of Butter
	+ 1 Cupful of Milk
	+ 3 Cupfuls of Flour
	+ 1 Teaspoon of Cream of Tartar
	+ 1 Tablespoon of Molasses
	+ A little Salt and flavor, Lemon or Almond
	+ 1 Large Cupful of Raisins
	+ 1/4 Pound of Citron
	+ 1 Teaspoon of Cinnamon and Cloves
	+ A little Nutmeg
	+ 1/2 Teaspoon of Soda
* **Instructions:**
	1. Cream the butter and sugar together, and add the well-beaten eggs and the milk. Mix the salt, soda and cream of tartar, with the flour. Stir all together.
	2. Put half of this mixture into two oblong pans. To the remainder add one tablespoonful of molasses, one large cupful of raisins, stoned and chopped, a 

**Roll Jelly Cake**

* **Ingredients:**
	+ 4 Eggs
	+ 1 Cupful of Sugar
	+ 1 Cupful of Flour
	+ 1/2 Teaspoonful of Soda
	+ Pinch of Salt
	+ 1 Teaspoonful of Cream of Tartar
	+ 1 Teaspoonful of Extract of Lemon
* **Instructions:**
	1. Beat together eggs and sugar, add salt and extract.
	2. Stir into the dry flour the soda and cream of tartar. Mix all together.
	3. Bake in a moderate oven, in a large pan, and turn out, when done, on a clean towel, which has been sprinkled with powdered sugar. Spread with jelly and roll while warm.

**Silver Cake**

* **Ingredients:**
	+ 1 Cupful of Sugar
	+ 1/2 Cupful of Butter
	+ 2 Cupfuls of Flour
	+ Whites of 3 Eggs
	+ 1/2 Teaspoonful of Soda
	+ 1/2 Cupful of Milk
	+ 1 Scant Teaspoonful of Cream of Tartar
	+ Almond Flavoring
* **Instructions:**
	1. Cream together the butter and sugar, add milk and flavoring.
	2. Stir cream of tartar and soda into dry flour. Last of all add whites of eggs, beaten to a 

Unfortunately, I couldn't find a specific recipe for a chocolate cake. If you'd like, I can try to provide you with a recipe for a different type of dessert, such as a chocolate pudding or a molasses candy. Let me know!

In [95]:
def rag_system(query,index,metadata,system_prompt,k=5):

    results = query_embeddings(query,index,metadata,k)

    combined_content = combined_retrived_content(results)

    response = generate_response(query,combined_content,system_prompt)

    return response

In [98]:
query1 = 'i want something meat '
response = rag_system(query1,index,metadata,system_prompt3)
display(Markdown(response))

Here are a few meat-based recipes I was able to find:

**Recipe 1: A La Mode Beef**

* **Recipe Title:** A La Mode Beef
* **Cuisine Type:** Not specified
* **Dish Type:** Meat dish
* **Tags:** Beef, Meat, Oven-baked
* **Ingredients:**
	+ 3 Pounds of Beef
	+ 6 Onions
	+ Potatoes
	+ Salt
	+ 4 or 5 White Turnips
* **Instructions:**
	1. Take three pounds of a cheap cut of beef. Wash, put into an iron pan, sprinkle over it salt to taste.
	2. Pare six onions, more or less, according to size of family, and prepare four or five small white turnips sliced thin.
	3. Lay these around the meat, and pour over all a quart of cold water. Put into the oven and bake three hours.
	4. Pare potatoes enough for the family, putting them in an hour and a half before serving.
	5. As the water cooks away, add more gravy, with flour wet with water, as you would with any roast meat.

**Recipe 2: Beefsteak Pie**

* **Recipe Title:** Beefsteak Pie
* **Cuisine Type:** Not specified
* **Dish Type:** Meat dish
* **Tags:** Beef, Meat, Oven-baked
* **Ingredients:**
	+ 2 Pounds of Beef (any cheap cut will do)
	+ 1 Onion
	+ 1 Tablespoonful of Salt
* **Instructions:**
	1. Cut the meat into small pieces; cover with cold water, salt and put into the oven; cut the onion into 

**Recipe 3: New England Boiled Dinner**

* **Recipe Title:** New England Boiled Dinner
* **Cuisine Type:** American (New England)
* **Dish Type:** Main Course, Stew/Boiled Dinner
* **Tags:** Traditional, Hearty, Comfort Food
* **Ingredients:**
	+ Corned beef
	+ White potatoes
	+ Sweet potatoes
	+ Cabbage
	+ Beets
	+ Turnips
	+ Squash
	+ Parsnips
	+ Carrots
* **Instructions:**
	1. Boil the meat four hours, the beets three hours, the cabbage one and a half hours, squash and turnips three-quarters of an hour.
	2. Boil these in one kettle, all together. Beets, carrots and parsnips should be boiled with the skin on.
	3. Pare the potatoes, pare and slice the squash and turnip.
	4. Pick the outer leaves from cabbage and cut in quarters.
	5. When done, pare parsnips and carrots. Drop the beets into cold water and slip the skin off with the hand.

**Recipe 4: Brunswick Stew**

* **Recipe Title:** Brunswick Stew
* **Cuisine Type:** American (Southern)
* **Dish Type:** Stew
* **Tags:** Hearty, Game Meat, Southern Classic
* **Ingredients:**
	+ 1 Chicken or 3 Pounds of Lamb
	+ 1 Onion
	+ 4 Potatoes
	+ 4 Ears of Corn
	+ Salt and Pepper
	+ 6 Tomatoes
* **Instructions:**
	1. Cook the chicken or lamb until tender in two quarts of water.
	2. Take from the water and chop fine.
	3. Put back in the liquor, add the corn, cut from the cob, tomatoes, onion, and potatoes all chopped.

**Recipe 5: Beef Stew with Dumplings**

* **Recipe Title:** Beef Stew with Dumplings
* **Cuisine Type:** Traditional/American
* **Dish Type:** Stew
* **Tags/Metadata:** comfort food, homemade, classic recipe
* **Ingredients:**
	+ 3 Pounds of Shin-bone with Meat
	+ 2 Large Onions
	+ 1 Tablespoon of Salt
	+ 6 Potatoes
	+ For Dumplings:
		- 2 Heaping Cupfuls of Flour
		- 2 Teaspoonfuls of Cream of Tartar
		- 1 Teaspoonful of Baking Soda
		- 1 Teaspoonful of Lard
		- 1/2 Teaspoonful of Salt
		- 1 Glass of Water
* **Step-by-Step Instructions:**
	1. Wash the meat, put into a kettle, cover with cold water and boil four hours. Add the salt, and more water, as it boils away. Pare the onions, wash and slice thin; put them in with the boiling meat, allowing two hours for cooking. Pare potatoes, wash, slice thin; put